In [2]:
from data import TIMITDatasetMelSpecdb, get_dataloader, LibriSpeechDataset
import torch
import torchaudio
from model import Model
from torchmetrics.functional import char_error_rate, word_error_rate
from modules.decoders.ctc import greedy_decoder
from quantizer import Quantizer
from typing import Dict
import pickle
import os

/home/shibutani/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
librispeech_train_dataset = LibriSpeechDataset(
    root="./datasets",
    split="train"
)
len(librispeech_train_dataset)

28539

In [3]:
with open("librispeech_quantized_indices_memory.pkl", "rb") as f:
            quantized_indices_memory = pickle.load(f)

In [3]:
# quantization of librispeech data
librispeech_train_dataset = LibriSpeechDataset(
    root="./datasets",
    split="train"
)
quantizer = Quantizer(DEVICE)
quantized_indices_memory = {}
for idx in range(len(librispeech_train_dataset)):
    # show progress
    if idx % 100 == 0:
        print(f"{idx / len(librispeech_train_dataset) * 100:.2f}%")
    audio = librispeech_train_dataset[idx][-2].unsqueeze(0).to(DEVICE)
    quantized_indices = quantizer.quantize(audio)
    quantized_indices_memory[idx] = quantized_indices[0].tolist()
with open("librispeech_quantized_indices_memory.pkl", "wb") as f:
    pickle.dump(quantized_indices_memory, f)

0.00%
0.35%
0.70%
1.05%
1.40%
1.75%
2.10%
2.45%
2.80%
3.15%
3.50%
3.85%
4.20%
4.56%
4.91%
5.26%
5.61%
5.96%
6.31%
6.66%
7.01%
7.36%
7.71%
8.06%
8.41%
8.76%
9.11%
9.46%
9.81%
10.16%
10.51%
10.86%
11.21%
11.56%
11.91%
12.26%
12.61%
12.96%
13.32%
13.67%
14.02%
14.37%
14.72%
15.07%
15.42%
15.77%
16.12%
16.47%
16.82%
17.17%
17.52%
17.87%
18.22%
18.57%
18.92%
19.27%
19.62%
19.97%
20.32%
20.67%
21.02%
21.37%
21.72%
22.08%
22.43%
22.78%
23.13%
23.48%
23.83%
24.18%
24.53%
24.88%
25.23%
25.58%
25.93%
26.28%
26.63%
26.98%
27.33%
27.68%
28.03%
28.38%
28.73%
29.08%
29.43%
29.78%
30.13%
30.48%
30.83%
31.19%
31.54%
31.89%
32.24%
32.59%
32.94%
33.29%
33.64%
33.99%
34.34%
34.69%
35.04%
35.39%
35.74%
36.09%
36.44%
36.79%
37.14%
37.49%
37.84%
38.19%
38.54%
38.89%
39.24%
39.59%
39.95%
40.30%
40.65%
41.00%
41.35%
41.70%
42.05%
42.40%
42.75%
43.10%
43.45%
43.80%
44.15%
44.50%
44.85%
45.20%
45.55%
45.90%
46.25%
46.60%
46.95%
47.30%
47.65%
48.00%
48.35%
48.71%
49.06%
49.41%
49.76%
50.11%
50.46%
50.81%
51.16%


In [13]:
librispeech_train_dataset[11][-1]

'AND HIS BEST SUIT OF CLOTHES WHICH WAS PLAIN PROOF THAT HE WAS GOING OUT OF AVONLEA AND HE HAD THE BUGGY AND THE SORREL MARE WHICH BETOKENED THAT HE WAS GOING A CONSIDERABLE DISTANCE NOW WHERE WAS MATTHEW CUTHBERT GOING AND WHY WAS HE GOING THERE'

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_dataset = TIMITDatasetMelSpecdb("test")
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False, drop_last=False, collate_fn=test_dataset.collate_fn)

In [ ]:
with open("quantized_indices_memory.pkl", "rb") as f:
    quantized_indices_memory = pickle.load(f)
len(quantized_indices_memory)

In [ ]:
#quantizer = Quantizer(DEVICE)
#quantized_indices_memory = {}
#for idx in range(len(train_dataset)):
#    # show progress
#    if idx % 100 == 0:
#        print(f"{idx / len(train_dataset) * 100:.2f}%")
#    audio = train_dataset[idx][-1].unsqueeze(0).to(DEVICE)
#    quantized_indices = quantizer.quantize(audio)
#    quantized_indices_memory[idx] = quantized_indices[0].tolist()
#with open("quantized_indices_memory.pkl", "wb") as f:
#    pickle.dump(quantized_indices_memory, f)

In [ ]:
def calculate_kl_divergence_between_uniform_and_empirical_distribution(sampled_quantized_idx_count: torch.Tensor):
    assert sampled_quantized_idx_count.ndim == 1
    # update sampled_quantized_idx_count
    # calculate KL divergence
    empirical_distribution = sampled_quantized_idx_count / sampled_quantized_idx_count.sum()
    uniform_distribution = torch.ones_like(empirical_distribution) / len(empirical_distribution)
    kl_divergence = torch.sum(empirical_distribution * torch.log(empirical_distribution / uniform_distribution))
    if kl_divergence < 0:
        raise ValueError("KL divergence must be positive.")
    return kl_divergence

In [ ]:
sampled_quantized_idx_count = torch.randint(0, 200, (320*320,), dtype=torch.float32)
sampled_quantized_idx_count += 1e-8
calculate_kl_divergence_between_uniform_and_empirical_distribution(sampled_quantized_idx_count)

In [ ]:
class KLBasedSampler:
    def __init__(self, quantized_indices_memory: Dict, dataset: torch.utils.data.Dataset, ratio: float, device: torch.device):
        self.device = device
        self.dataset = dataset
        self.ratio = ratio
        self.target_num_samples = int(len(dataset) * ratio)
        self.initial_num_samples =int(0.1 * self.target_num_samples)
        self.sampled_indices = set()
        self.not_sampled_indices = set(range(len(dataset)))
        self.sampled_quantized_idx_count = torch.zeros(320 * 320, dtype=torch.float32)
        self.quantized_indices_memory = quantized_indices_memory
        # sample initial samples
        self.sample_initial_samples()
    
    def calculate_kl_divergence_between_uniform_and_empirical_distribution(self, sample):
        """Calculate KL divergence between uniform and empirical distribution.
        Args:
            sample (1D torch.Tensor): quantized indices tensor
        Returns:
            kl_divergence (float): KL divergence between uniform and empirical distribution.
        """
        assert sample.ndim == 1
        # update sampled_quantized_idx_count
        sampled_quantized_idx_count = self.sampled_quantized_idx_count.clone()
        with torch.no_grad():
            for quantized_idx in sample:
                sampled_quantized_idx_count[quantized_idx] += 1
        # calculate KL divergence
        # avoid zero division
        sampled_quantized_idx_count += 1e-8
        empirical_distribution = sampled_quantized_idx_count / sampled_quantized_idx_count.sum()
        uniform_distribution = torch.ones_like(empirical_distribution) / len(empirical_distribution)
        kl_divergence = torch.sum(empirical_distribution * torch.log(empirical_distribution / uniform_distribution))
        if kl_divergence < 0:
            raise ValueError("KL divergence must be positive.")
        return kl_divergence

    def sample_initial_samples(self):
        # sample initial samples based on random sampling
        initial_indices = torch.randperm(len(self.dataset))[:self.initial_num_samples].tolist()
        self.sampled_indices.update(initial_indices)
        self.not_sampled_indices.difference_update(initial_indices)
        # update sampled_quantized_idx_count
        with torch.no_grad():
            for idx in initial_indices:
                quantized_indices = torch.tensor(self.quantized_indices_memory[idx])
                for quantized_idx in quantized_indices:
                    self.sampled_quantized_idx_count[quantized_idx] += 1

    def sample(self):
        # sample new samples based on KL divergence until the number of samples reaches the target number of samples
        # return subset of dataset
        while len(self.sampled_indices) < self.target_num_samples:
            # show progress
            print(f"sampled {len(self.sampled_indices) / self.target_num_samples * 100:.2f} %")
            # calculate KL divergence for each not sampled indices
            kl_divergences = {}
            not_sampled_indices = list(self.not_sampled_indices)
            for idx in not_sampled_indices:
                quantized_indices = torch.tensor(self.quantized_indices_memory[idx])
                kl_divergence = self.calculate_kl_divergence_between_uniform_and_empirical_distribution(quantized_indices)
                kl_divergences[idx] = kl_divergence
            # select the index with the minimum KL divergence
            min_kl_divergence_idx = min(kl_divergences.keys(), key=kl_divergences.get)
            print(f"min_kl_divergence: {kl_divergences[min_kl_divergence_idx]}")
            # update sampled_indices and not_sampled_indices
            self.sampled_indices.add(min_kl_divergence_idx)
            self.not_sampled_indices.remove(min_kl_divergence_idx)
            # update sampled_quantized_idx_count
            quantized_indices = self.quantized_indices_memory[min_kl_divergence_idx]
            for quantized_idx in quantized_indices:
                self.sampled_quantized_idx_count[quantized_idx] += 1

        # return subset of self.dataset
        return torch.utils.data.Subset(self.dataset, list(self.sampled_indices))


In [ ]:
train_dataset = TIMITDatasetMelSpecdb("train")

In [ ]:
kl_based_sampler = KLBasedSampler(quantized_indices_memory=quantized_indices_memory, dataset=train_dataset, ratio=0.1, device=DEVICE)
subset = kl_based_sampler.sample()


In [ ]:
import pickle
# save subset
with open("subset.pkl", "wb") as f:
    pickle.dump(subset, f)

In [ ]:
def random_sampler(dataset: torch.utils.data.Dataset, ratio: float):
    """Randomly sample a subset of the dataset"""
    num_samples = int(len(dataset) * ratio)
    indices = torch.randperm(len(dataset))[:num_samples].tolist()
    return torch.utils.data.Subset(dataset, indices)

In [ ]:
random_train_subset = random_sampler(train_dataset, 0.8)
random_train_dataloader = get_dataloader(random_train_subset, batch_size=32, shuffle=True, drop_last=True, collate_fn=train_dataset.collate_fn)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This Learning is running on {DEVICE}")

In [ ]:
input_feature_size = 80
self_attn_dim = 256
feed_forward_dim = 1024
token_size = len(train_dataset.vocab.keys())
num_epochs = 40
total_size = len(train_dataset)

In [ ]:
from modules.preprocessing.subsampling import Conv2DSubSampling
from modules.transformers.encoder import TransformerEncoder
from modules.transformers.scheduler import TransformerLR
from torch import nn


class Model(nn.Module):
    def __init__(self, nlabel):
        super(Model, self).__init__()
        self.nlabel = nlabel
        # out_sizeがSelf-Attentionの入力次元
        self.conv2d_sub_sampling = Conv2DSubSampling(
            in_size=80, out_size=256, kernel_size1=3, kernel_size2=3, stride1=2, stride2=1)
        # in_hidden_sizeがFFの次元
        self.transformer_encoder = TransformerEncoder(
            in_size=256, nlayer=12, nhead=4, in_hidden_size=1024, dropout=0.1, norm_first=True)
        self.fc = nn.Linear(256, nlabel, bias=True)
        self.log_softmax = nn.functional.log_softmax

    def forward(self, x, x_lengths):
        # args:
        #   x: [B, T, in_size]
        #   x_lengths: [B]
        #       padding前のシーケンス長
        # return:
        #   log_prob: [B, T, nlabel]
        #   y_lengths: [B]
        #       非パディング部分のシーケンス長
        subsampled_x, subsampled_x_length = self.conv2d_sub_sampling(x, x_lengths)
        encoded, encoded_inner = self.transformer_encoder(
            subsampled_x, subsampled_x_length)  # [B, T', subsampled_in_size]
        y = self.fc(encoded)  # [B, T', nlabel]
        y_lengths = subsampled_x_length
        log_probs = self.log_softmax(y, dim=-1)  # [B, T', nlabel]
        return log_probs, y_lengths

In [ ]:

asr_model = Model(nlabel=token_size).to(DEVICE)
ctc_loss = torch.nn.CTCLoss(reduction="sum", blank=train_dataset.ctc_token_id)
optimizer = torch.optim.Adam(asr_model.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9)
scheduler = TransformerLR(optimizer, d_model=256, warmup_steps=8000) # Warmup終了時点でおよそ0.0017になっている

for epoch in range(num_epochs):
    asr_model.train()
    train_epoch_loss = 0
    train_epoch_cer = 0
    train_epoch_wer = 0
    train_cnt = 0

    for i, (bidx, bx, bx_len, by, by_len, texts, phonetic_details) in enumerate(random_train_dataloader):
        bx = bx.to(DEVICE)
        bx_len = bx_len.to(DEVICE)
        by = by.to(DEVICE)
        by_len = by_len.to(DEVICE)
        optimizer.zero_grad()
        log_probs, y_lengths = asr_model(bx, bx_len)
        loss = ctc_loss(log_probs.transpose(1, 0), by, y_lengths, by_len)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_epoch_loss += (loss.item() / bx.size(0))

        # calculate CER
        hypothesis = torch.argmax(log_probs, dim=-1)
        hypotheses = greedy_decoder(hypothesis, train_dataset.vocab, "[PAD]", "|", "_")
        answers = greedy_decoder(by, train_dataset.vocab, "[PAD]", "|", "_")
        train_epoch_cer += char_error_rate(hypotheses, answers)
        train_epoch_wer += word_error_rate(hypotheses, answers)

        train_cnt += 1

    print(f"Epoch {epoch + 1} of {num_epochs} train loss: {train_epoch_loss / train_cnt}, train CER: {train_epoch_cer / train_cnt}, train WER: {train_epoch_wer / train_cnt}")

    asr_model.eval()
    test_epoch_loss = 0
    test_epoch_cer = 0
    test_epoch_wer = 0
    test_cnt = 0
    with torch.no_grad():
        for i, (bidx, bx, bx_len, by, by_len, texts, phonetic_details) in enumerate(test_dataloader):
            bx = bx.to(DEVICE)
            bx_len = bx_len.to(DEVICE)
            by = by.to(DEVICE)
            by_len = by_len.to(DEVICE)
            log_probs, y_lengths = asr_model(bx, bx_len)
            loss = ctc_loss(log_probs.transpose(1, 0), by, y_lengths, by_len)
            test_epoch_loss += (loss.item() / bx.size(0))

            # calculate CER
            hypothesis = torch.argmax(log_probs, dim=-1)
            hypotheses = greedy_decoder(hypothesis, train_dataset.vocab, "[PAD]", "|", "_")
            answers = greedy_decoder(by, train_dataset.vocab, "[PAD]", "|", "_")
            test_epoch_cer += char_error_rate(hypotheses, answers)
            test_epoch_wer += word_error_rate(hypotheses, answers)

            test_cnt += 1
        
    print(f"Epoch {epoch + 1} of {num_epochs} test loss: {test_epoch_loss / test_cnt}, test CER: {test_epoch_cer / test_cnt}, test WER: {test_epoch_wer / test_cnt}")
